In [1]:
import matplotlib.pyplot as plt
import scipy.constants as cons
import pandas as pd
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from qutip import *
from funcs import *
from scipy.optimize import curve_fit

def straightline(x, A, B):
    return A * x + B

In [ ]:
%matplotlib

This notebook is kind of a mess but should be understandable, may need to change the files being read in below.
If anything unclear please ask.

Ps. Don't run all cells at once as there are many graphs being plotted.

In [25]:
# inncubicdfmax

In [19]:
# Reading in data from file
# cubicdf = pd.read_csv('cubic_good3.csv')
cubicdf = pd.concat([pd.read_csv('cubic_good3.csv'), 
                     pd.read_csv('cubic_good2.csv')], ignore_index=True)
cubicdf = cubicdf[cubicdf.nmean < 6.05].sort_values('nmean')

# cubicdf = pd.read_csv('cubicanalytic_good3.csv').sort_values('nmean')
# cubicdf = cubicdf[cubicdf.nmean < 6.05].sort_values('nmean')

# pd.concat([pd.read_csv(), pd.read_csv()], ignore_index=True)

# inncubicdf = pd.read_csv('inner_cubic_highdensity.csv')
inncubicdf = pd.concat([pd.read_csv('inner_cubic_highdensity.csv'), 
                        pd.read_csv('inner_cubic_nmeanregion.csv'),
                        pd.read_csv('inner_cubic.csv')], ignore_index=True)
inncubicdf = inncubicdf[inncubicdf.nmean < 6.05].sort_values('nmean')

In [ ]:
# inncubicdf.describe()
# cubicdf.describe()

In [ ]:
# fig, ax = myplotwigner(innercubic(0.290678, 0.027966, 150)[0], [-10,10], [-10,10], fineness = 100)
# plt.show()

In [ ]:
fig, ax = my3dscatter(cubicdf.gamma, cubicdf.r, cubicdf.WLN, title='', axeslabels=['gamma', 'r', 'WLN'])

plt.show()

In [91]:
fig, ax = my3dscatter(inncubicdf.gamma, inncubicdf.r, inncubicdf.WLN, title='', axeslabels=['gamma', 'r', 'WLN'])

plt.show()

In [10]:
# Creating bins for nmean data
bins1 = np.linspace(-0.05, cubicdf.nmean.max(), 60)
bins2 = np.linspace(-0.05, inncubicdf.nmean.max(), 60)

cubicdf['nmeanbins'] = pd.cut(cubicdf.nmean, bins1)
inncubicdf['nmeanbins'] = pd.cut(inncubicdf.nmean, bins2)

In [11]:
# Obtain the maximum Mana in each nmean bin
maxWLNs1 = []
maxWLNs2 = []
for bin in np.unique(cubicdf.nmeanbins):
    maxWLNs1.append(cubicdf[cubicdf.nmeanbins == bin].WLN.max())

    
for bin in np.unique(inncubicdf.nmeanbins):
    maxWLNs2.append(inncubicdf[inncubicdf.nmeanbins == bin].WLN.max())

# Define new dataframes as those with gamma, r coords corresponding to max Mana in nmeanbins 
cubicdfmax = cubicdf[cubicdf.WLN.isin(maxWLNs1)]
# cubicdfmax = cubicdfmax[cubicdfmax.nmean > 0.15]
inncubicdfmax = inncubicdf[inncubicdf.WLN.isin(maxWLNs2)]

In [12]:
# Generate Mana Data for Fock states to compare
fnmeans = []
fWLNs = []
N = 10
for i in range(1,7):
    fockstate = fock(N, i)
    fnmeans.append(expect(num(N), fockstate))
    fWLNs.append(wln(fockstate, 1e-5)[0])

In [18]:
datafile = 'triple_photon_max_data.csv'
with open(datafile, 'a') as f:
            inncubicdfmax.to_csv(f, index=False)
        
datafile = 'cubic_max_data.csv'
with open(datafile, 'a') as f:
            cubicdfmax.to_csv(f, index=False)

In [14]:
# Plot nmean agains max Mana
fig = plt.figure()
ax = plt.axes()
ax.plot(cubicdfmax.nmean, cubicdfmax.WLN, label='Cubic Phase State')
ax.scatter(cubicdfmax.nmean, cubicdfmax.WLN, label='Cubic Phase State')
ax.plot(inncubicdfmax.nmean, inncubicdfmax.WLN, label='Inner Cubic State')
ax.scatter(inncubicdfmax.nmean, inncubicdfmax.WLN, label='Inner Cubic State')
ax.scatter(fnmeans, fWLNs, label='Fock States')

ax.set_title('''Maximal Value of Mana (WLN) against the mean value of the number 
operator for the inner cubic state and cubic phase state''')
ax.set_xlabel('<n>')
ax.set_ylabel('Mana / WLN')
ax.legend()

plt.show()

In [21]:
# Calculate least squares fit to the Max Mana data
[A1, B1], pcov1 = curve_fit(straightline, cubicdfmax.gamma, cubicdfmax.r)
perr1 = np.sqrt(np.diag(pcov1))

[A2, B2], pcov2 = curve_fit(straightline, inncubicdfmax.gamma, inncubicdfmax.r)
perr2 = np.sqrt(np.diag(pcov2))

In [23]:
# Plot Points at which we find the maximum Mana for constant nmean for each state

plt.figure()
ax = plt.axes()
ax.set_title('Cubic Phase State')
ax.set_xlabel('Gamma')
ax.set_ylabel('r, Squeezing Factor')
# Lines
plt.plot(cubicdfmax.gamma, straightline(cubicdfmax.gamma, A1, B1))
plt.plot(cubicdfmax.gamma, straightline(cubicdfmax.gamma, A1 + perr1[0], B1 + perr1[1]))
plt.plot(cubicdfmax.gamma, straightline(cubicdfmax.gamma, A1 - perr1[0], B1 - perr1[1]))
# Scatters
plt.scatter(cubicdfmax.gamma, cubicdfmax.r)
plt.scatter(cubicdf.gamma, cubicdf.r, s = 1)
plt.show()


plt.figure()
ax = plt.axes()
ax.set_title('Inner Cubic State')
ax.set_xlabel('Gamma')
ax.set_ylabel('r, Squeezing Factor')
# Lines
plt.plot(inncubicdfmax.gamma, straightline(inncubicdfmax.gamma, A2, B2))
plt.plot(inncubicdfmax.gamma, straightline(inncubicdfmax.gamma, A2 + perr2[0], B2 + perr2[1]))
plt.plot(inncubicdfmax.gamma, straightline(inncubicdfmax.gamma, A2 - perr2[0], B2 - perr2[1]))
# Scatters
plt.scatter(inncubicdfmax.gamma, inncubicdfmax.r)
plt.scatter(inncubicdf.gamma, inncubicdf.r, s = 1)
plt.show()

In [ ]:
# Same as above but 3d scatters

fig, ax = my3dscatter(cubicdfmax.gamma, cubicdfmax.r, cubicdfmax.WLN, 'Cubic Phase State', ['gamma', 'r', 'WLN'])

ax.scatter(cubicdfmax.gamma, cubicdfmax.r, cubicdfmax.WLN, s=50, c=cubicdfmax.WLN, cmap=cm.jet)

ax.scatter(cubicdf.gamma, cubicdf.r, cubicdf.WLN, s = 1, c='black')

plt.show()

In [68]:
fig, ax = my3dscatter(inncubicdfmax.gamma, inncubicdfmax.r, inncubicdfmax.WLN, 
                      'Inner Cubic State', ['gamma', 'r', 'WLN'])

ax.scatter(inncubicdfmax.gamma, inncubicdfmax.r, inncubicdfmax.WLN, s=50, c=inncubicdfmax.WLN, cmap=cm.jet)

ax.scatter(inncubicdf.gamma, inncubicdf.r, inncubicdf.WLN, s = 1, c='black')

plt.show()